## Derive theoretical phase speed of Diabatic Rossby wave

- Refers to Equations 7 and 8 in the latest paper draft

In [13]:
import numpy as np
import xarray as xr
import metpy.calc as mpcalc
from scipy import signal
import scipy.stats as st
%matplotlib inline

### Define the latitude and longitude bounds for our calculations

- Section 5.3 in the paper
- Focus on the model level closest to 7 km 
- Define a latitude-longitude strip over which to calculate terms
- Read in potential vorticity (`PV`) separately (5.5 to 9.0º rather than 5.5 to 7.0º)

In [14]:
lon_start = 104.0
lon_end = 115.0
lat_start = 5.5
lat_end = 7.0

height_level = 7000

### Read in N768 MetUM data at 12 UTC on 23 October 2018 (T+48)

- There is a 12-h discrepancy between the time-string in the file and the validity time it represents
- This is why we subtract 12 when defining the integer `time`

In [15]:
time = int(48)-12
gl_pe='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pe{0:03d}.nc'.format(time)
gl_pc='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pc{0:03d}.nc'.format(time)
gl_pb='/nobackup/earshar/borneo/case_20181021T1200Z_N768_v2/umglaa_pb{0:03d}.nc'.format(time)

input_data_pe = xr.open_dataset(gl_pe).metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                                        earth_radius=6371229.0)
input_data_pc = xr.open_dataset(gl_pc, drop_variables=['unspecified_5','unspecified_6',   # heating rate 
                                                       'unspecified_9','unspecified_10'])
input_data_pc = input_data_pc.metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                               earth_radius=6371229.0)
input_data_pb = xr.open_dataset(gl_pb).metpy.assign_crs(grid_mapping_name='latitude_longitude', 
                                                        earth_radius=6371229.0)

### Focus on specified latitude-longitude strip

In [16]:
gdata_pe=input_data_pe.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end),
                                 longitude_1=slice(lon_start, lon_end), latitude_1=slice(lat_start, lat_end) )
gdata_pb=input_data_pb.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end) )
gdata_pc=input_data_pc.sel(longitude=slice(lon_start, lon_end), latitude=slice(lat_start, lat_end) )

### Calculate sum of terms contributing to diabatic heating rate

- convection
- radiation
- friction
- etc

In [17]:
heating_rate = 0
vars_list = ['unspecified', 'unspecified_1', 'unspecified_2', 'unspecified_3',
             'unspecified_4', 'unspecified_7','unspecified_8']
for var in vars_list:
    heating_rate = heating_rate + gdata_pc[var]

### Tidy data and interpolate to reference set of vertical levels

In [18]:
ht_coords = gdata_pe.u['hybrid_ht_1'].data.astype('int32')

u_gl = gdata_pe.u.interp(longitude_1=gdata_pe.v["longitude"],
                         hybrid_ht_1=ht_coords,
                         method="linear").assign_coords(height_levels=("hybrid_ht_1",
                        ht_coords)).swap_dims({"hybrid_ht_1":
                        "height_levels"})
v_gl = gdata_pe.v.interp(latitude_1=gdata_pe.u["latitude"],
                         hybrid_ht_1=ht_coords,
                         method="linear").assign_coords(height_levels=("hybrid_ht_1",
                        ht_coords)).swap_dims({"hybrid_ht_1":
                                               "height_levels"})
pv_gl = gdata_pe.field83.interp(hybrid_ht=ht_coords,
                                method="linear").assign_coords(height_levels=("hybrid_ht",
                                ht_coords)).swap_dims({"hybrid_ht":
                                                       "height_levels"})
w_gl  = gdata_pe.dz_dt.interp(hybrid_ht=ht_coords,
                              method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})

q_gl  = gdata_pb.q.interp(hybrid_ht=ht_coords,
                          method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})
th_gl = gdata_pb.theta.interp(hybrid_ht=ht_coords,
                              method="linear").assign_coords(height_levels=("hybrid_ht",
                            ht_coords)).swap_dims({"hybrid_ht":
                                                   "height_levels"})
rho_gl = gdata_pb.field27.interp(hybrid_ht_1=ht_coords,
                                 method="linear").assign_coords(height_levels=("hybrid_ht_1",
                                ht_coords)).swap_dims({"hybrid_ht_1":"height_levels"})
heating_rate = heating_rate.interp(hybrid_ht=ht_coords,
                                   method="linear").assign_coords(height_levels=("hybrid_ht",
                                ht_coords)).swap_dims({"hybrid_ht":"height_levels"})
exner_func = gdata_pb.field7.interp(hybrid_ht_1=ht_coords,
                                 method="linear").assign_coords(height_levels=("hybrid_ht_1",
                                ht_coords)).swap_dims({"hybrid_ht_1":"height_levels"})

### Reduce the variable dimensions from 3D to 2D

In [19]:
u_gl = u_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
v_gl = v_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
w_gl = w_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))

th_gl = th_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
q_gl = q_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
rho_gl = rho_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
ertel_pv = pv_gl.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end)) # * 1000000.

vort_gl = mpcalc.vorticity(u_gl, v_gl, dx=None, dy=None)
abs_vort_gl = mpcalc.absolute_vorticity(u_gl, v_gl, dx=None, dy=None)

vort_gl = vort_gl.sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
abs_vort_gl = abs_vort_gl.sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))
heating_rate = heating_rate.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end)) / 3600.
exner_func = exner_func.squeeze('t').sel(latitude=slice(lat_start,lat_end)).sel(longitude=slice(lon_start,lon_end))

## Upper wave phase speed calculation

### Calculate first term in Eq. 8 (absolute vorticity)

In [21]:
rho_upper = rho_gl.sel(height_levels=slice(8000,10000)).mean(dim=['height_levels'])
rho_lower = rho_gl.sel(height_levels=slice(2000,4000)).mean(dim=['height_levels'])
rho_ave = ( (rho_upper + rho_lower) / 2).mean() 

q_ave = q_gl.sel(height_levels=height_level, method="nearest").mean()
abs_vort = abs_vort_gl.sel(height_levels=height_level, method="nearest").mean()

th_upper = th_gl.sel(height_levels=slice(8000,10000)).mean(dim=['height_levels'])
th_lower = th_gl.sel(height_levels=slice(2000,4000)).mean(dim=['height_levels'])
d_theta  = (th_upper - th_lower).mean()
th_ref   = 300.0
dz       = 6000.0

term_1 = (abs_vort * d_theta / (rho_ave * dz) ) # Current value = 3.52 x 10-7
term_1

<xarray.DataArray ()>
<Quantity(3.04210709e-07, '1 / second')>
Coordinates:
    t              datetime64[ns] 2018-10-23T12:00:00
    metpy_crs      object Projection: latitude_longitude
    hybrid_ht_1    int32 7036
    height_levels  int32 7036

### Calculate second term in Eq. 8 (moist stability)

In [23]:
th_dz = mpcalc.first_derivative(th_gl, axis=0)
th_dz = th_dz * (9.81 / th_ref)
moist_stability = q_gl * th_dz

moist_stability_upper = moist_stability.sel(height_levels=slice(8000,
                            10000)).mean(dim=['height_levels']).mean()
moist_stability_lower = moist_stability.sel(height_levels=slice(2000,
                            4000)).mean(dim=['height_levels']).mean()
moist_stability_interface = moist_stability.sel(height_levels=height_level, method='nearest').mean()

term_2 = (moist_stability_upper - moist_stability_lower) / moist_stability_interface # Current value = -2.49

moist_stratification_factor = term_1 * term_2 # Current value = -8.77 x 10-7
moist_stratification_factor

<xarray.DataArray ()>
<Quantity(-1.3623389e-06, '1 / second')>
Coordinates:
    t              datetime64[ns] 2018-10-23T12:00:00
    metpy_crs      object Projection: latitude_longitude
    hybrid_ht_1    int32 7036
    height_levels  int32 7036
    hybrid_ht      int32 7036

### Set up Gaussian kernel to smooth the datasets for the wave phase speed calculations

In [24]:
def create_gaussian_kernel(kernel_length: int, nsig: float): 
    '''
    returns a 2D Gaussian kernel with side-length (for use in convolution)
    stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy
    http://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm
    '''
    x = np.linspace(-nsig, nsig, kernel_length+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()

kernel_2d = create_gaussian_kernel(3, 1.0)

### Calculate third term in Eq. 8 (diabatic heating)

- Calculate the amplitude of input fields by smoothing using a Gaussian convolution function, then taking the standard deviation
- Definition using standard deviation assumes that all of the variance is explained by the wave

In [25]:
exner_func_upper = exner_func.sel(height_levels=height_level, method="nearest").mean()
heating_rate_upper = (heating_rate.sel(height_levels=height_level, method="nearest"))
ertel_pv_upper = ertel_pv.sel(height_levels=height_level, method='nearest')

heating_rate_smoothed = xr.DataArray(signal.convolve2d(heating_rate_upper, kernel_2d, boundary='symm', mode='same'))
heating_rate_mean = heating_rate_smoothed.mean()
heating_rate_std = (heating_rate_smoothed - heating_rate_mean).std()

ertel_pv_smoothed = xr.DataArray(signal.convolve2d(ertel_pv_upper, kernel_2d, boundary='symm', mode='same'))
ertel_pv_mean = ertel_pv_smoothed.mean()
ertel_pv_std = (ertel_pv_smoothed - ertel_pv_mean).std()

pv_wave_length = 8.0 * 111000 
pv_wave_number = (2 * np.pi) / pv_wave_length

term_3a = 1 / pv_wave_number
term_3b = heating_rate_std / d_theta 

term_3c = 1 / ertel_pv_std

term_3 = (term_3a * term_3b * term_3c).mean()

upper_wave_phase_speed = moist_stratification_factor * term_3
upper_wave_phase_speed

<xarray.DataArray ()>
<Quantity(-1.32512765, '1 / second')>
Coordinates:
    t              datetime64[ns] 2018-10-23T12:00:00
    metpy_crs      object Projection: latitude_longitude
    hybrid_ht_1    int32 7036
    height_levels  int32 7036
    hybrid_ht      int32 7036

## Calculate additional term estimating the impact of basic-state PV gradients

- Double check the height at which we're calculating this term (should be 7 km)
- Double check that the extent of the x-y coordinates are correct too (should be 5.5 to 9.0º N)
- Double check the units in all calculations (especially PV)

### Magnitude of the full balanced v-wind from the SGT tool 

In [26]:
v_gl_6km = (v_gl.sel(height_levels=height_level, method="nearest"))

v_gl_smoothed = xr.DataArray(signal.convolve2d(v_gl_6km, kernel_2d, boundary='symm', mode='same'))
v_gl_mean = v_gl_smoothed.mean()
v_gl_std = (v_gl_smoothed - v_gl_mean).std()

### Calculate the phase speed contribution from the meridional gradient of (smoothed) PV

In [31]:
gdata_pe_pv = input_data_pe.sel(longitude=slice(104.0, 115.0), latitude=slice(5.5, 9.0) )

pv_upper = gdata_pe_pv.field83.interp(hybrid_ht=ht_coords,
                                method="linear").assign_coords(height_levels=("hybrid_ht",
                                ht_coords)).swap_dims({"hybrid_ht":
                                                        "height_levels"})

pv_upper = pv_upper.squeeze('t').sel(height_levels=height_level, method="nearest")

pv_ave_north = pv_upper.sel(latitude=9.0, method="nearest").mean() # 1.14 x 10^-7
pv_ave_south = pv_upper.sel(latitude=5.5, method="nearest").mean() # 3.35 x 10^-7

pv_dy = (pv_ave_north - pv_ave_south) / (3.5 * 111000)
pv_gradient_term = (pv_dy * v_gl_std) / (pv_wave_number * ertel_pv_std)
pv_gradient_term

<xarray.DataArray ()>
array(-0.89483851)
Coordinates:
    t              datetime64[ns] 2018-10-23T12:00:00
    metpy_crs      object Projection: latitude_longitude
    hybrid_ht      int32 7036
    height_levels  int32 7036

## Lower wave phase speed calculation

In [32]:
th_lower = th_gl.sel(height_levels=1400, method="nearest")

gdata_pb_theta = input_data_pb.sel(longitude=slice(104.0, 115.0), latitude=slice(5.5, 9.0) )

th_lower = gdata_pb_theta.theta.interp(hybrid_ht=ht_coords,
                                     method="linear").assign_coords(height_levels=("hybrid_ht",
                                    ht_coords)).swap_dims({"hybrid_ht":"height_levels"})
                                    
th_lower = th_lower.squeeze('t').sel(height_levels=1400, method="nearest")

th_ave_north = th_lower.sel(latitude=9.0, method="nearest").mean()
th_ave_south = th_lower.sel(latitude=5.5, method="nearest").mean()

th_dy = (th_ave_north - th_ave_south) / (3.5 * 111000)

In [33]:
theta_wave_length = 8.0 * 111000
theta_wave_number = (2 * np.pi) / theta_wave_length

theta_smoothed = xr.DataArray(signal.convolve2d(th_lower, kernel_2d, boundary='symm', mode='same'))
theta_mean = theta_smoothed.mean()
theta_std = (theta_smoothed - theta_mean).std()

v_lower = v_gl.sel(height_levels=1400, method="nearest")
v_smoothed = xr.DataArray(signal.convolve2d(v_lower, kernel_2d, boundary='symm', mode='same'))
v_mean = v_smoothed.mean()
v_std = (v_smoothed - v_mean).std()

base_state_gradient = - ( (1 / theta_wave_number) * (th_dy / d_theta) )
wave_amplitude = d_theta / theta_std
frequency = theta_wave_number * v_std
length_scale = 1 / theta_wave_number
lower_wave_phase_speed = base_state_gradient * wave_amplitude * frequency * length_scale

### Print out the upper and lower wave phase speed values

In [34]:
upper_wave_phase_speed, lower_wave_phase_speed

(<xarray.DataArray ()>
 <Quantity(-1.32512765, '1 / second')>
 Coordinates:
     t              datetime64[ns] 2018-10-23T12:00:00
     metpy_crs      object Projection: latitude_longitude
     hybrid_ht_1    int32 7036
     height_levels  int32 7036
     hybrid_ht      int32 7036,
 <xarray.DataArray ()>
 array(-1.5810024)
 Coordinates:
     t              datetime64[ns] 2018-10-23T12:00:00
     metpy_crs      object Projection: latitude_longitude
     hybrid_ht      int32 1396
     height_levels  int32 1396)

### Calculate the mean meridional wind over the duration of the event

- Do this separately in `borneo_vortex_drw_phase_speed_mean_state.ipynb`
- Eventually build the functionality into this notebook